In [ ]:
!git clone https://github.com/acmarkes/traffic-sign-recog.git
%cd /content/traffic-sign-recog
!pip install gdown

import gdown
imgs = 'https://drive.google.com/uc?id=17an3cKGO1I2WlBPSa6zcAucL-jISSHEn'
labels = 'https://drive.google.com/uc?id=17gFHTQvU0utSNm3XMv2I8Ud4IzuqDLTl'

gdown.download(imgs, 'procTrainImages.joblib', quiet=False)
gdown.download(labels, 'train_labels.joblib', quiet=False)


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

import utils

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
procTrainImages = joblib.load('procTrainImages.joblib')
print(procTrainImages.shape)
utils.sample_images(procTrainImages, seed_num=42)
trainLabels = joblib.load('train_labels.joblib')
print(len(trainLabels))


In [ ]:
img_tweaks = {
    'featurewise_center':True,
    'featurewise_std_normalization':True,
    'rotation_range':20,
    'width_shift_range':0.2,
    'height_shift_range':0.2,
}

In [ ]:
Xy_train, Xy_val = utils.get_dataset_partitions_tf(procTrainImages, trainLabels, gen_kws=img_tweaks)


In [ ]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides,
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1,
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides,
                                    padding="same", use_bias=False),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[32, 32, 1],
                              padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64

for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(43, activation="softmax"))


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(Xy_train, batch_size=32 ,epochs=3, validation_data=Xy_val, callbacks=[callback])
